In [ ]:
import cv2
import numpy as np
from mtcnn import MTCNN
from keras_facenet import FaceNet
from numpy.linalg import norm


In [ ]:
detector = MTCNN()
embedder = FaceNet()


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))


In [ ]:
ref_img = cv2.imread("ref.jpg")
ref_img = cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB)

faces = detector.detect_faces(ref_img)
x, y, w, h = faces[0]['box']
ref_face = ref_img[y:y+h, x:x+w]
ref_face = cv2.resize(ref_face, (160,160))

ref_embedding = embedder.embeddings([ref_face])[0]


In [ ]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    try:
        faces = detector.detect_faces(rgb)
    except:
        faces = []

    for face in faces:
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        if w < 40 or h < 40:
            continue

        face_img = rgb[y:y+h, x:x+w]
        if face_img.size == 0:
            continue

        face_img = cv2.resize(face_img, (160,160))

        emb = embedder.embeddings([face_img])[0]
        sim = cosine_similarity(ref_embedding, emb)

        label = "Matched" if sim > 0.7 else "Unknown"
        color = (0,255,0) if sim > 0.7 else (0,0,255)

        cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
        cv2.putText(frame, f"{label} {sim:.2f}",
                    (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
